In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
hoi_data = pd.read_csv('./Health_Opportunity_Index_20231031.csv')
hoi_data.head()

,Census Tract,Rural~Urban,Access to Care,Employment Accessibility,Affordability,Air Quality,Population Churning,Education,Food Accessibility,Income Inequality,Job Participation,Population Density,Segregation,Material Deprivation,Walkability,Community Environment Profile,Consumer Opportunity Profile,Economic Opportunity Profile,Wellness Disparity Profile,Health Opportunity Index
0,51650011400,Urban,0.486042,0.015251,0.043043,0.730782,0.468193,0.630734,0.977517,0.034818,0.150,0.043334,0.770141,0.000000,0.245825,0.540012,0.390787,0.006034,0.409863,0.000000
1,51760040300,Urban,0.502987,0.042804,0.641642,0.351528,0.022901,0.871575,0.933302,0.082321,0.373,0.227476,0.639438,0.133458,0.307647,0.777319,0.764695,0.033064,0.488495,0.015558
2,51013101701,Urban,0.432198,0.131224,0.627628,0.800711,0.513995,0.853716,0.933302,0.395412,0.759,1.000000,0.801044,0.343863,0.731265,1.000000,0.827284,0.326435,0.355932,0.032861
3,51710002500,Urban,0.465814,0.041683,0.021021,0.754040,0.444020,0.697862,0.741324,0.258974,0.436,0.113065,0.919320,0.228861,0.321052,0.561143,0.302549,0.183497,0.319484,0.077873
4,51760040400,Urban,0.502987,0.084305,0.662663,0.382158,0.427481,0.800714,0.933302,0.000000,0.551,0.231267,0.822681,0.184284,0.326225,0.719178,0.773519,0.075911,0.395098,0.085121


In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [110]:
svm_pipeline = Pipeline([('std_scaler', StandardScaler()), ('svm', OneClassSVM(kernel='rbf', gamma='auto', nu=0.01))])
updated_hoi_data = hoi_data.drop(["Census Tract", "Health Opportunity Index", "Population Density", 'Community Environment Profile', 'Consumer Opportunity Profile', 'Economic Opportunity Profile', 'Wellness Disparity Profile'], axis=1)
updated_hoi_data = pd.get_dummies(updated_hoi_data, drop_first=True)
final_hoi = svm_pipeline.fit(updated_hoi_data)
outlier_index_list = np.argwhere(final_hoi.predict(updated_hoi_data) == -1).flatten().tolist()
len(outlier_index_list)

60

In [109]:
svm_reduc_pipeline = Pipeline([('std_scaler', StandardScaler()), ('svm', OneClassSVM(kernel='rbf', gamma='auto', nu=0.01))])
reduc_hoi_data = hoi_data[['Community Environment Profile', 'Consumer Opportunity Profile', 'Economic Opportunity Profile', 'Wellness Disparity Profile']]
reduc_hoi_data = pd.get_dummies(reduc_hoi_data, drop_first=True)
final_reduc_hoi = svm_pipeline.fit(updated_hoi_data)
reduced_outlier_index_list = np.argwhere(final_reduc_hoi.predict(reduc_hoi_data) == -1).flatten().tolist()
len(reduced_outlier_index_list)

47

In [118]:
red_set = set(reduced_outlier_index_list)
out_set = set(outlier_index_list)
same_outlier = list(red_set.intersection(out_set))

In [119]:
hoi_data.iloc[same_outlier]

,Census Tract,Rural~Urban,Access to Care,Employment Accessibility,Affordability,Air Quality,Population Churning,Education,Food Accessibility,Income Inequality,Job Participation,Population Density,Segregation,Material Deprivation,Walkability,Community Environment Profile,Consumer Opportunity Profile,Economic Opportunity Profile,Wellness Disparity Profile,Health Opportunity Index
0,51650011400,Urban,0.486042,0.015251,0.043043,0.730782,0.468193,0.630734,0.977517,0.034818,0.150,0.043334,0.770141,0.000000,0.245825,0.540012,0.390787,0.006034,0.409863,0.000000
3,51710002500,Urban,0.465814,0.041683,0.021021,0.754040,0.444020,0.697862,0.741324,0.258974,0.436,0.113065,0.919320,0.228861,0.321052,0.561143,0.302549,0.183497,0.319484,0.077873
1669,51013101900,Urban,0.455237,0.120780,0.737738,0.608152,0.787532,0.906435,0.933302,0.539406,0.771,0.081548,0.280161,0.567297,0.375462,0.423415,0.811294,0.322796,0.637758,0.530258
102,51700030100,Urban,0.496763,0.072211,0.000000,0.728580,0.564885,0.650867,0.898084,0.305263,0.572,0.043289,0.066475,0.085685,0.268508,0.449310,0.251218,0.214953,0.776117,0.274571
136,51089011200,Rural,0.224278,0.086415,0.314314,0.890727,0.889313,0.613520,0.453030,0.273549,0.526,0.003638,0.709461,0.325489,0.013109,0.280285,0.171224,0.273923,0.255181,0.295750
40,51027010100,Rural,0.181837,0.004241,0.063063,0.911786,0.917303,0.639062,0.951248,0.299865,0.465,0.000697,0.823801,0.323340,0.063456,0.305606,0.415973,0.152325,0.166810,0.224555
234,51185020500,Rural,0.505154,0.009875,0.274274,0.892335,0.851145,0.642433,0.604491,0.312955,0.455,0.006206,0.614337,0.456783,0.134436,0.292411,0.339949,0.211883,0.502826,0.322964
1486,51167030401,Urban,0.461949,0.024953,0.562563,0.946471,0.926209,0.701516,0.942139,0.316059,0.564,0.003945,0.063811,0.480101,0.083732,0.223630,0.649698,0.184005,0.752790,0.494916
1295,51820003100,Urban,0.897720,0.157948,0.313313,0.612937,0.791349,0.661616,0.933302,0.442105,0.568,0.023458,0.355525,0.262334,0.313801,0.402433,0.426380,0.313842,0.913098,0.470818
1874,51179010201,Urban,0.467134,0.696429,0.304304,0.850762,0.039440,0.812955,0.942139,0.907557,0.857,0.000426,0.820064,0.697936,0.070880,0.393656,0.142578,0.867737,0.371010,0.728815


,Census Tract,Access to Care,Employment Accessibility,Affordability,Air Quality,Population Churning,Education,Food Accessibility,Income Inequality,Job Participation,Population Density,Segregation,Material Deprivation,Walkability,Community Environment Profile,Consumer Opportunity Profile,Economic Opportunity Profile,Wellness Disparity Profile,Health Opportunity Index
Census Tract,1.00,-0.00,0.14,-0.33,-0.26,-0.32,-0.14,-0.13,-0.17,-0.07,0.08,-0.07,-0.33,0.33,0.33,-0.31,-0.03,0.05,-0.32
Access to Care,-0.00,1.00,0.23,0.12,-0.23,-0.08,0.20,0.07,0.06,0.21,0.13,-0.01,0.02,0.19,0.18,0.17,0.19,0.71,0.27
Employment Accessibility,0.14,0.23,1.00,0.21,-0.14,-0.16,0.28,0.07,0.37,0.58,0.24,0.01,0.14,0.34,0.24,0.06,0.78,0.15,0.38
Affordability,-0.33,0.12,0.21,1.00,0.10,0.21,0.65,0.27,0.36,0.37,0.09,0.06,0.62,0.08,-0.12,0.84,0.29,0.04,0.60
Air Quality,-0.26,-0.23,-0.14,0.10,1.00,0.27,-0.11,0.08,0.22,-0.12,-0.29,0.04,0.32,-0.41,-0.60,0.04,0.11,-0.19,0.31
Population Churning,-0.32,-0.08,-0.16,0.21,0.27,1.00,-0.11,0.12,0.14,-0.04,-0.36,0.01,0.39,-0.40,-0.68,0.21,0.00,-0.06,0.52
Education,-0.14,0.20,0.28,0.65,-0.11,-0.11,1.00,0.23,0.17,0.45,0.26,0.03,0.45,0.34,0.26,0.80,0.20,0.12,0.35
Food Accessibility,-0.13,0.07,0.07,0.27,0.08,0.12,0.23,1.00,0.12,0.14,0.03,0.04,0.22,0.04,-0.04,0.56,-0.05,0.02,0.24
Income Inequality,-0.17,0.06,0.37,0.36,0.22,0.14,0.17,0.12,1.00,0.42,0.03,0.06,0.40,-0.02,-0.24,0.16,0.79,-0.00,0.62
Job Participation,-0.07,0.21,0.58,0.37,-0.12,-0.04,0.45,0.14,0.42,1.00,0.32,0.05,0.11,0.35,0.28,0.29,0.73,0.11,0.40
